# Cortex Compliance AI - Fine-Tuning with Phi-2

Uses Microsoft Phi-2 (2.7B) - small enough to train without quantization.

**No bitsandbytes/triton issues!**

## Steps:
1. Runtime → Change runtime type → **T4 GPU**
2. Run all cells

In [ ]:
# Step 1: Install dependencies (simple - no bitsandbytes needed)
!pip install -q transformers==4.41.0 accelerate peft trl datasets huggingface_hub sentencepiece

print("✅ Installed! Run all cells (no restart needed)")

In [ ]:
# Step 2: Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Step 3: Load dataset from HuggingFace (already uploaded!)
from datasets import load_dataset

dataset = load_dataset("maaninder/cortex-compliance-data", split="train")
print(f"✅ Loaded {len(dataset)} training examples from HuggingFace")
print(dataset[0]["text"][:200] + "...")

In [ ]:
# Step 4: Load Phi-2 model (2.7B - fits on T4 without quantization)
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
import warnings
warnings.filterwarnings("ignore")

MODEL_NAME = "microsoft/phi-2"  # 2.7B params, fits on T4 GPU

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

# Add LoRA
model = get_peft_model(model, LoraConfig(
    r=16, lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "dense"],
    lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
))

print("✅ Phi-2 loaded with LoRA")
model.print_trainable_parameters()

In [ ]:
# Step 5: Train
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    args=TrainingArguments(
        output_dir="./output",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        optim="paged_adamw_8bit",
        save_strategy="no",
        report_to="none",
    ),
)

print("🚀 Training started...")
trainer.train()
print("✅ Training complete!")

In [ ]:
# Step 6: Save to HuggingFace Hub
from huggingface_hub import notebook_login
notebook_login()

model.push_to_hub("maaninder/cortex-compliance-phi2")
tokenizer.push_to_hub("maaninder/cortex-compliance-phi2")
print("✅ Model saved to: https://huggingface.co/maaninder/cortex-compliance-phi2")

In [ ]:
# Step 7: Test the model
test_prompt = "### Instruction:\nGenerate a Personal Data Processing Policy for ООО Тест (INN: 1234567890)\n\n### Response:\n"
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

model.eval()
outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.7, do_sample=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Done!

Model: https://huggingface.co/maaninder/cortex-compliance-phi2

**Why Phi-2:**
- 2.7B params (fits on T4 without quantization)
- No bitsandbytes/triton needed
- Still very capable for document generation